In [1]:
import tkinter as tk
import tkinter.ttk as ttk
import re
import numpy as np

import nbimporter
import preprocessing
from publish_subscribe import Reader
from publish_subscribe import listToString
from tkinter import *

In [2]:
%store -r new_df
%store -r author_list
%store -r word_list
%store -r syn
%store -r author_key

In [3]:
class MainPanel:
    def __init__(self, reader, publisher,author_list,word_list,syn,author_key):
        """
        初始化：
        1. reader 订阅者
        2. publisher 发布者
        3. author_list 诗人集合
        4. word_list 关键词集合dict，value为出现该关键词的诗歌编号
        5. syn 关键词-近似词dict
        6. author_key 诗人-关键词dict
        """
        self.reader = reader
        self.publisher = publisher
        self.all_authors = author_list
        self.all_words = word_list
        self.cur_author = ""
        self.con = dict()
        self.subs = []
        self.mylist =None
        self.syn = syn
        self.author_key = author_key
        
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('400x600')
        self.root.title("古诗订阅")
        self.root.iconbitmap('cat.ico')
        self.root.configure(bg='#546FA7')
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12),bg='#546FA7',fg='white').pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), bg='#CCF1FB',command=self.add_subscription).pack(pady=40)
        self.add_button1 = tk.Button(self.root, text="查询", font=(None,12),width=5, bg='#CCF1FB',command=self.check).pack(pady=20)
        self.root.mainloop()
        
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.title("新增订阅")
        self.add_panel.iconbitmap('cat.ico')
        self.add_panel.configure(bg='#546FA7')
        #present_author_label = tk.Label(self.add_panel, text=f"已订阅作者：{self.reader.authors}", font=(None,10)).pack(pady=20)
        label_1 = tk.Label(self.add_panel, text="诗人:", font=(None, 10), bg='#546FA7',fg='white').pack(pady=20)
        self.new_author_entry = tk.Entry(self.add_panel)
        self.new_author_entry.pack()
        
        label_2 = tk.Label(self.add_panel, text="关键词:", font=(None, 10), bg='#546FA7',fg='white').pack(pady=20)
        self.key_word_entry = tk.Entry(self.add_panel)
        self.key_word_entry.pack()

        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10),bg='#CCF1FB', command=self.check_new).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", bg='#546FA7',fg='white',font=(None, 10))
        self.hint_label.pack()
    
    def check(self):
        """
        用户点击"查询"按钮，显示查询诗人关键词的界面
        """
        self.add_panel2 = tk.Tk()
        self.add_panel2.geometry('400x1000')
        self.add_panel2.title("查询诗人和关键词")
        self.add_panel2.iconbitmap('cat.ico')
        self.add_panel2.configure(bg='#546FA7')
        #present_author_label = tk.Label(self.add_panel, text=f"已订阅作者：{self.reader.authors}", font=(None,10)).pack(pady=20)
        label_1 = tk.Label(self.add_panel2, text="诗人:", font=(None, 10), bg='#546FA7',fg='white').pack(pady=20)
        self.check_authorkey_entry = tk.Entry(self.add_panel2)
        self.check_authorkey_entry.pack()
        confirm_button = tk.Button(self.add_panel2, text="关键词查询", font=(None, 10),bg='#CCF1FB', command=self.check_authorkey).pack(pady=20)
        check_authorlist = tk.Button(self.add_panel2, text="查询诗人列表", font=(None, 10),bg='#CCF1FB', command=self.printauthorlist).pack(pady=20)
        self.hint_label1 = tk.Label(self.add_panel2, text="", bg='#546FA7',fg='white',font=(None, 10))
        self.hint_label1.pack()
        self.sb = Scrollbar(self.add_panel2)
        self.sb.pack(side=RIGHT, fill = Y)
        self.keyaut = Listbox(self.add_panel2, width=55, height=20, bg='#546FA7',fg='white',yscrollcommand=self.sb.set)
        self.add_panel2.mainloop()
    
    def printauthorlist(self):
        """
        用户点击"查询诗人列表"按钮，显示诗人列表
        """
        self.add_1 = tk.Tk()
        self.add_1.geometry('400x300')
        self.add_1.title("查询诗人列表")
        self.add_1.iconbitmap('cat.ico')
        self.add_1.configure(bg='#546FA7')
        tmp = []
        sb = Scrollbar(self.add_1)
        sb.pack(side=RIGHT, fill = Y)
        autlist = Listbox(self.add_1, width=1000, height=1000, bg='#546FA7',fg='white',yscrollcommand=sb.set)
        for i in range(len(self.all_authors)):
            tmp.append(self.all_authors[i])
            if len(tmp) == 5:
                autlist.insert(END, tmp)
                tmp = []
        autlist.pack(side=LEFT ,fill=BOTH)
        sb.config(command=autlist.yview)
        
        self.add_1.mainloop()

    def check_authorkey(self):
        """
        用户查询诗人的关键词集合，以listbox方式显示该关键词集合
        """
        authorname = self.check_authorkey_entry.get()
        self.remove_text()
        if len(authorname)> 0 and authorname not in self.all_authors:
            self.hint_label1.config(text=f"并无诗人{authorname}，任何疑问请查询诗人列表",bg='#546FA7',fg='white')
        else:
            
            tmp = []
            
            self.hint_label1.config(text=f"诗人{authorname}的关键词有:",bg='#546FA7',fg='white')
            k = self.author_key[authorname]
            
            for i in range(len(k)):
                tmp.append(k[i])
                if i != 0 and i % 10 ==0:
                    self.keyaut.insert(END, tmp)
                    tmp = []
            self.keyaut.pack(side=LEFT ,fill=BOTH)
            self.sb.config(command=self.keyaut.yview)
#             self.au = tk.Label(self.add_panel2, text=tmp, bg='#546FA7',fg='white',font=(None, 10))
#             self.au.pack()
#             self.reset = Button(self.add_panel2, text="重置")
#             self.reset['command']= (lambda b = self.reset, k =True:self.remove_text(b,k))
#             self.reset.pack()
            
    def remove_text(self):
        """
        用以当用户反复查询诗人的关键词集合时，删除先前显示的诗人关键词集合
        """
        self.keyaut.delete(0, END)
        self.hint_label1.config(text="",bg='#546FA7',fg='white')
        
        
    def check_new(self):
        """
        用户点击"确认"按钮后，检查订阅是否合法
        分为6种：
            1. 并无此诗人
            2. 并无此关键词
            3. 当用户只输入诗人名（并无关键词时）若该诗人在并无关键词的情况下也已被订阅，订阅失败
            4. 当用户只输入关键词（并无诗人名时）若该关键词在并无诗人名的情况下也已被订阅，订阅失败
            5. 该诗人+关键词已被订阅，订阅失败
            6. 有此诗人但该诗人并无此关键词
            else 订阅成功！ 
        
        """
        new_author = self.new_author_entry.get()
        key_word = self.key_word_entry.get()
        check_keyword = 0
        sentences = key_word.split(' ')
        should_proceed = True
        if len(new_author)> 0 and new_author not in self.all_authors:
            self.hint_label.config(text=f"并无诗人{new_author}，不支持订阅",bg='#546FA7',fg='white')
            should_proceed = False
        if len(key_word)> 0:
            error_word = ""
            ## check error word
            k = sentences.copy()
            for i in k:
                if i not in self.all_words:
                    if check_keyword >=1:
                        error_word+=" "
                    error_word += i
                    check_keyword+=1
                    sentences.remove(i)
            if check_keyword >= 1:
                should_proceed = False
                self.hint_label.config(text=f"并无该{error_word}关键词，不支持订阅",bg='#546FA7',fg='white')
        if len(new_author)> 0 and len(key_word) == 0 and new_author in self.reader.poems and should_proceed == True:
            should_proceed = False
            self.hint_label.config(text=f"该作者{new_author}已订阅，请重新输入",bg='#546FA7',fg='white')
        if len(key_word)> 0 and len(new_author) == 0 and key_word in self.reader.poems and should_proceed == True:
            should_proceed = False
            self.hint_label.config(text=f"该关键词{key_word}已订阅，请重新输入",bg='#546FA7',fg='white')
        if len(new_author)> 0 and len(key_word) > 0 and should_proceed == True:
            check = new_author
            check+=" "
            check+=key_word
            if(check in self.reader.poems):
                should_proceed = False
                self.hint_label.config(text=f"该诗人{new_author}的关键词{key_word}已订阅，请重新输入",bg='#546FA7',fg='white')
            else:
                senten = key_word.split(" ")
                no_words =[]
                for i in senten:
                    if i not in self.author_key[new_author]:
                        no_words.append(i)
                if len(no_words)>0:
                    should_proceed = False
                    tmp = ' '.join(no_words)
                    self.hint_label.config(text=f"该诗人{new_author}并无{tmp}关键词，请重新输入",bg='#546FA7',fg='white')
        if should_proceed == True:
            if len(new_author)> 0 and len(key_word) > 0:
                self.subs.append([new_author,key_word])
            elif len(new_author)== 0 and len(key_word) > 0:
                self.subs.append(["",key_word])
            elif len(new_author)> 0 and len(key_word) == 0:
                self.subs.append([new_author,""])
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tmp_tk.title("订阅成功")
            tmp_tk.configure(bg='#546FA7')
            tk.Label(tmp_tk, text="订阅成功", font=(None, 20),bg='#546FA7',fg='white').pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12),bg='#CCF1FB', command=tmp_tk.destroy).pack(pady=20)
            self.reader.subscribeToPublisher(new_author, self.publisher,key_word)
            self.add_panel.destroy()
            self.update_main_panel()
    
    def printList(self, event):
        """
        删除订阅时，判断cursor选择要删除的是哪个订阅
        """
        self.to_del =  str(self.mylist.get(self.mylist.curselection()))
        return
    
    def remove_subscription(self):
        """
        用户点击"删除订阅"按钮，显示已有的订阅，选择要删除的订阅
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.title("删除订阅")
        self.add_panel.iconbitmap('cat.ico')
        self.add_panel.configure(bg='#546FA7')
        
        #present_author_label = tk.Label(self.add_panel, text=f"已订阅作者：{self.reader.authors}", font=(None,10)).pack(pady=20)
#         label_1 = tk.Label(self.add_panel, text="诗人:", font=(None, 10), bg='#546FA7',fg='white').pack(pady=20)
#         self.new_author_entry = tk.Entry(self.add_panel)
#         self.new_author_entry.pack()
        instr = tk.Label(self.add_panel, text="删除订阅", font=(None,10),bg='#546FA7',fg='white').pack(pady=20)
        sb = Scrollbar(self.add_panel)
        sb.pack(side=RIGHT, fill = Y)
        self.mylist = Listbox(self.add_panel, width=35, height=20,bg='#CCF1FB',yscrollcommand=sb.set)
        
        for keywords in self.reader.poems:
            self.mylist.insert(END, keywords)
        
        self.mylist.pack(side=LEFT, fill=BOTH)   
        sb.config(command=self.mylist.yview)
        self.mylist.bind('<<ListboxSelect>>',self.printList)
#         self.to_del = str(self.mylist.get(ANCHOR))
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10),bg='#CCF1FB', command=self.check_delete).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", bg='#546FA7',fg='white',font=(None, 10))
        self.hint_label.pack()

        self.add_panel.mainloop()
            
        
    def check_delete(self):
        """
        用户在删除订阅界面，点击"确认"按钮后，执行删除订阅
        
        """
#         print(f"ori author list{self.reader.authors}")
#         print(f"ori poem{self.reader.poems}")
        tmp_tk = tk.Tk()
        tmp_tk.geometry("300x200")
        tmp_tk.title("删除成功")
        tmp_tk.configure(bg='#546FA7')
        tk.Label(tmp_tk, text="删除成功", font=(None, 20),bg='#546FA7',fg='white').pack(pady=20)
        tk.Button(tmp_tk, text="确认", font=(None, 12),bg='#CCF1FB', command=tmp_tk.destroy).pack(pady=20)
        cnt = 0
        for key_word in self.reader.poems:
            if(key_word == self.to_del):
                break
            cnt+=1 
        tmp =self.subs[cnt]
        del self.subs[cnt]
        self.reader.unsubscribeToPublisher(tmp[0],self.publisher,tmp[1])
        self.add_panel.destroy()
        self.update_main_panel()

            
    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面，增加了删除订阅按钮
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12),bg='#546FA7',fg='white').pack(pady=20)
        cnt = 1
        for key_word in self.reader.poems:
            button = tk.Button(self.root, text=("订阅"+str(cnt)+": "+key_word), width=25, font=(None,12),bg='#CCF1FB')
            button.bind("<Button-1>", lambda event, arg=key_word: self.show_contents(event,arg))
            button.pack(pady=10)
            cnt+=1
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12),bg='#CCF1FB', command=self.add_subscription).pack(pady=40)
        self.remove_button = tk.Button(self.root, text="删除订阅", font=(None,12),bg='#CCF1FB', command=self.remove_subscription).pack()
        self.add_button1 = tk.Button(self.root, text="查询", font=(None,12),width=5, bg='#CCF1FB',command=self.check).pack(pady=20)
        self.root.update()
    
    def search(self, text_widget, keyword, tag):
        """
        用户查询已订阅的诗歌，高亮关键词
        """
        pos = '1.0'
        while True:
            idx = text_widget.search(keyword, pos, END)
            if not idx:
                break
            pos = '{}+{}c'.format(idx, len(keyword))
            text_widget.tag_add(tag, idx, pos)
        
    def show_contents(self, event, key_word):
        """
        用户查询已订阅的诗歌
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('470x650')
        self.add_panel.iconbitmap('cat.ico')
        self.add_panel.configure(bg='#546FA7')
        self.add_panel.title(f"{key_word} ")
        titles = []
        authorss = []
        cnt = 0
        
        for k in self.reader.poems:
            if(key_word == k):
                break
            cnt+=1 
        the_key1 = self.subs[cnt]
        the_key2 = the_key1[1]
#         print(f"hi{key_word}")
        for i in self.reader.poems[key_word]:
            titles.append(i.title)
            authorss.append(i.name)
            self.con[i.title]=listToString(i.content)
        total = len(titles)
        num_tabs = 0
        if total%2 == 1:
            num_tabs = 1
        num_tabs += total//2
#         print(f"prep for tabs{key_word}")
        tab_main=ttk.Notebook(self.add_panel, width=2000, height=1000)#创建分页栏
        tabs = [None]*num_tabs
        cnt = int(0)
        for i in range(0,num_tabs):
            tabs[i] = tk.Frame(self.add_panel,bg='#546FA7')
            tab_main.add(tabs[i], text=str(i+1))
            if (len(the_key1[0])!=0):
                tk.Label(tabs[i], text=f"诗人: {the_key1[0]}",font=(None, 12),bg='#546FA7',fg='white').pack()
            if(len(the_key2) != 0):
                sentences = the_key2.split(" ")
                tk.Label(tabs[i], text=f"关键词: {the_key2}",font=(None, 12),bg='#546FA7',fg='white').pack()
                for a in sentences:
                    tmp = ""
                    if a in self.syn and len(self.syn[a])>0:
                        for b in self.syn[a]:
                            tmp+=b
                            tmp+=" "
                        tk.Label(tabs[i], text=f"关键词: {a}的近似词为: {tmp}",bg='#546FA7',fg='white').pack()
                    else:
                        tk.Label(tabs[i], text=f"关键词: {a}无近似词",bg='#546FA7',fg='white').pack()
            t = Text(tabs[i],highlightthickness = 0, borderwidth=0,background='#546FA7')
            t.pack()
            t.configure(state="normal")
            t.tag_config('a', background='#CCF1FB',foreground = 'black',justify = 'center')
            t.tag_config('b', background='#546FA7',foreground = 'white',justify = 'center')
            t.tag_lower('b')
            for j in range(0,2):
                if(cnt+j>=total):
                    break
                tmp = " "
                tmp += titles[cnt+j]
                tmp += "\n "
                tmp += authorss[cnt+j]
                tmp +='\n '
                for a in self.con[titles[cnt+j]]:
                    tmp+=a
                    if a == ',':
                        tmp+='\n '
                t.insert(END, "\n\n\n\n",'b')
                t.insert(END, tmp+"。",'b')
                if(len(the_key2) != 0):
                    sentences = the_key2.split(" ")
                    for a in sentences:
                        self.search(t, a,'a')
                        if a in self.syn:
                            for b in self.syn[a]:
                                self.search(t, b,'a')   
            t.configure(state="disabled")
#                 tk.Label(tabs[i], text=tmp, font=(None, 15),bg='#546FA7',fg='white').pack(pady=(20 if j == 0 else 70))
            cnt+=2        
        tab_main.pack(side=tk.TOP)
        self.add_panel.mainloop()